In [11]:
import pandas as pd
import arcpy
import os
sp = 'Хадыженское'

NP_plan = f'np_plan'
NP = f'np'
ZU = 'zu'
FZ = f'fz'
CAT = f'lu'
DU = f'DU_empty'

fz_table_xls = r"\\junkart\FS\1_ГРАД_ПРОЕКТЫ\23_Краснодарский_край\2_ГП\Апшеронский_Хадыженское\5_Работа\Для баланса.xlsx"

folder = arcpy.mp.ArcGISProject("CURRENT").homeFolder
xls_name = os.path.join(folder, fr'Переводы_{sp}.xlsx') ###ИТОГОВЫЙ ФАЙЛ

In [12]:
def table_to_data_frame(in_table, input_fields=None, where_clause=None):
    """Function will convert an arcgis table into a pandas dataframe with an object ID index, and the selected
    input fields using an arcpy.da.SearchCursor."""
    OIDFieldName = arcpy.Describe(in_table).OIDFieldName
    if input_fields:
        final_fields = [OIDFieldName] + input_fields
    else:
        final_fields = [field.name for field in arcpy.ListFields(in_table)]
    data = [row for row in arcpy.da.SearchCursor(in_table, final_fields, where_clause=where_clause)]
    fc_dataframe = pd.DataFrame(data, columns=final_fields)
    fc_dataframe = fc_dataframe.set_index(OIDFieldName, drop=True)
    return fc_dataframe

In [13]:
def keep_fields(layer, keep_set):
    fields = arcpy.ListFields(layer)
    field_set = set()
    for field in fields:
        field_set.update([field.name])
    return list(field_set - keep_set)

In [14]:
NP_TYPE = {1: 'Город',
           2: 'Поселок городского типа',
           3: 'Рабочий поселок',
           4: 'Дачный поселок',
           5: 'Курортный поселок',
           6: 'Городской поселок',
           7: 'Лесной поселок',
           8: 'Населенный пункт',
           9: 'Поселок',
           10: 'Село',
           11: 'Деревня',
           12: 'Хутор',
           13: 'Станция',
           14: 'Железнодорожная станция',
           15: 'Поселок при станции (поселок станции)',
           16: 'Поселок при железнодорожной станции',
           17: 'Железнодорожный разъезд',
           18: 'Железнодорожная казарма',
           19: 'Железнодорожная будка',
           20: 'Железнодорожный остановочный пункт',
           21: 'Остановочная платформа',
           22: 'Железнодорожный блокпост',
           23: 'Блокпост',
           24: 'Железнодорожная платформа',
           25: 'Железнодорожный путевой пост',
           26: 'Дорожный разъезд',
           27: 'Железнодорожная площадка',
           28: 'Остановочный пункт',
           29: 'Железнодорожная водокачка',
           30: 'Разъезд',
           31: 'Железнодорожный комбинат',
           32: 'Железнодорожный пост',
           33: 'Монтёрский пункт',
           34: 'Станица',
           35: 'Слобода',
           36: 'Местечко',
           37: 'Починок',
           38: 'Участок',
           39: 'Выселок',
           40: 'Дома',
           41: 'Кордон',
           42: 'Казарма',
           43: 'Заимка',
           44: 'Лесничество',
           45: 'Лесоучасток',
           46: 'Маяк',
           47: 'Остров',
           48: 'Мыс',
           49: 'Гидрологический пост',
           50: 'Метеостанция',
           51: 'Аал',
           52: 'Аул',
           53: 'Улус',
           54: 'Арбан',
           55: 'Зимовка',
           56: 'Дом отдыха',
           57: 'Турбаза',
           58: 'База отдыха',
           59: 'Санаторий',
           60: 'Усадьба',
           61: 'Центральная усадьба',
           62: 'Отдельный дом',
           63: 'Площадка',
           64: 'Погост',
           65: 'Подстанция',
           66: 'Карьер',
           67: 'Аэропорт',
           68: 'Будка',
           69: 'Контрольный пункт связи',
           70: 'Данных о типе населенного пункта нет'}

In [15]:
output_db = os.path.join(folder, 'Perevody.gdb')

In [16]:
if not arcpy.Exists(output_db):
    arcpy.management.CreateFileGDB(folder, 'Perevody.gdb')

In [17]:
###ЗА ГРАНИЦАМИ
arcpy.env.addOutputsToMap = False
arcpy.env.workspace = "CURRENT"
keep_fields_set = {"DU_TYPE", "CadNumber", "cat", "bydoc", "cost", "NAME", "SETTL_TYPE", "OBJECTID", "SHAPE", "SHAPE_Length", "SHAPE_Area", "Override", "Ext_Zone_Code", "Shape", "Shape_Length", "Shape_Area"}
keep_fields_set.update(['CLASSID'])
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.activeMap
for lyr in m.listLayers(CAT):
    lyr.definitionQuery = "STATUS = 2"
arcpy.analysis.Erase(CAT, NP, output_db + '\\MO_1', "")
arcpy.analysis.Erase(output_db + '\\MO_1', NP_plan, output_db + '\\MO', "")
arcpy.analysis.PairwiseIntersect((output_db + '\\MO', ZU), output_db + "\\MO_zu", "all", "", "input")
arcpy.management.DeleteField(output_db + "\\MO_zu", keep_fields(output_db + "\\MO_zu", keep_fields_set))
arcpy.analysis.Erase(output_db + '\\MO', ZU, output_db + '\\MO_nerazgr', "")
arcpy.management.Merge([output_db + '\\MO_nerazgr', output_db + "\\MO_zu"], output_db + "\\MO_ALL", "", "NO_SOURCE_INFO")
arcpy.analysis.PairwiseIntersect((output_db + "\\MO_ALL", FZ), output_db + "\\MO_FZ", "all", "", "input")
arcpy.management.DeleteField(output_db + "\\MO_FZ", keep_fields(output_db + "\\MO_FZ", keep_fields_set))
for lyr in m.listLayers(CAT):
    lyr.definitionQuery = "STATUS = 1 And (Note <> 'Двойной учет' Or Note IS NULL)"
arcpy.env.addOutputsToMap = True
arcpy.analysis.PairwiseIntersect((output_db + "\\MO_FZ", CAT), output_db + "\\MO_FINAL", "all", "", "input")
keep_fields_set.update(['CLASSID_1'])
arcpy.management.DeleteField(output_db + "\\MO_FINAL", keep_fields(output_db + "\\MO_FINAL", keep_fields_set))
for lyr in m.listLayers(DU):
    lyr.definitionQuery = "DU_TYPE IN ('После 2016/Нет информации', 'искл') And (Note <> 'Амнистия' Or Note IS NULL)"
arcpy.management.Dissolve(DU, output_db + '\\DU', ['DU_TYPE'])
# arcpy.analysis.PairwiseIntersect((output_db + "\\MO_preFINAL", output_db + '\\DU'), output_db + "\\MO_DU", "all", "", "input")
# arcpy.management.DeleteField(output_db + "\\MO_DU", keep_fields(output_db + "\\MO_DU", keep_fields_set))
# arcpy.analysis.Erase(output_db + "\\MO_preFINAL", output_db + "\\MO_DU", output_db + '\\MO_DU_nerazgr', "")
# arcpy.env.addOutputsToMap = True
# arcpy.management.Merge([output_db + "\\MO_DU", output_db + '\\MO_DU_nerazgr'], output_db + "\\MO_FINAL", "", "NO_SOURCE_INFO")    

<Result 'C:\\Users\\ya.shatalov\\Desktop\\Data\\1_Проекты\\Апшеронское\\Apsheronskoe\\Perevody.gdb\\DU'>

In [18]:
###ВКЛЮЧАЕМЫЕ ЗУ
keep_fields_set = {"DU_TYPE", "CadNumber", "cat", "bydoc", "cost", "NAME", "SETTL_TYPE", "OBJECTID", "SHAPE", "SHAPE_Length", "SHAPE_Area", "Override", "Ext_Zone_Code", "Shape", "Shape_Length", "Shape_Area"}
arcpy.env.addOutputsToMap = False
arcpy.env.workspace = "CURRENT"
arcpy.analysis.Erase(NP_plan, NP, output_db + '\\NP_vkl', "")
arcpy.analysis.PairwiseIntersect((NP_plan, NP), output_db + "\\NP_NP_plan_inter", "all", "", "input")
arcpy.management.CalculateField(output_db + "\\NP_NP_plan_inter", "SETTL_TYPE_1", "!SETTL_TYPE_1! + 100", "PYTHON3")
arcpy.management.DeleteField(output_db + '\\NP_vkl', keep_fields(output_db + '\\NP_vkl', keep_fields_set))
###ДЛЯ АППЕНДА ПОКА ЧТО ТАК
layer = output_db + "\\NP_NP_plan_inter"
target_layer = output_db + '\\NP_vkl'
prename = ''
check_fields = {} 
except_fields = ['SHAPE', 'Shape', 'Shape_Length', 'Shape_Area', 'Shape.STArea()', 'Shape.STLength()'] 
fields_input = arcpy.ListFields(layer)
fields_target = arcpy.ListFields(target_layer)
fieldMappings = arcpy.FieldMappings()
for field in fields_target:
    print(field.name)
    if field.name in except_fields:
        continue
    fldMap = arcpy.FieldMap()
    field_name = field.name[field.name.rfind('.')+1:]
    if field_name in check_fields.keys():
        fldMap.addInputField(layer, check_fields[field_name])
    elif field_name in [field2.name for field2 in fields_input]:
        fldMap.addInputField(layer, field_name)
    elif field_name in [field2.name[field2.name.rfind('.')+1:] for field2 in fields_input]:
        fldMap.addInputField(layer, prename + field_name)
    else:
        continue
    targetField = fldMap.outputField
    targetField.name, targetField.aliasName, targetField.type = field.name, field.aliasName, field.type
    fldMap.outputField = targetField
    fieldMappings.addFieldMap(fldMap)
###АПЕНД ТРЕБУЕТ ЖЕРТВ
arcpy.management.Append([output_db + "\\NP_NP_plan_inter"], output_db + '\\NP_vkl', "NO_TEST", fieldMappings, expression="NAME <> NAME_1")
arcpy.analysis.PairwiseIntersect((output_db + '\\NP_vkl', ZU), output_db + "\\NP_vkl_zu", "all", "", "input")
arcpy.management.DeleteField(output_db + "\\NP_vkl_zu", keep_fields(output_db + "\\NP_vkl_zu", keep_fields_set))
arcpy.analysis.Erase(output_db + '\\NP_vkl', output_db + "\\NP_vkl_zu", output_db + '\\NP_vkl_nerazgr', "")
arcpy.management.Merge([output_db + '\\NP_vkl_nerazgr', output_db + "\\NP_vkl_zu"], output_db + "\\NP_vkl_ALL", "", "NO_SOURCE_INFO")
arcpy.analysis.PairwiseIntersect((output_db + "\\NP_vkl_ALL", FZ), output_db + "\\NP_vkl_FZ", "all", "", "input")
arcpy.management.DeleteField(output_db + "\\NP_vkl_FZ", keep_fields(output_db + "\\NP_vkl_FZ", keep_fields_set))
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.activeMap
for lyr in m.listLayers(CAT):
    lyr.definitionQuery = "STATUS = 1 And (Note <> 'Двойной учет' Or Note IS NULL)"
arcpy.analysis.PairwiseIntersect((output_db + "\\NP_vkl_FZ", CAT), output_db + "\\NP_vkl_FINAL", "all", "", "input")
keep_fields_set.update(['CLASSID', 'CLASSID_1'])
arcpy.management.DeleteField(output_db + "\\NP_vkl_FINAL", keep_fields(output_db + "\\NP_vkl_FINAL", keep_fields_set))


# for lyr in m.listLayers(CAT):
#     lyr.definitionQuery = "STATUS = 2"
# arcpy.analysis.PairwiseIntersect((output_db + "\\NP_vkl_FZ_CAT", CAT), output_db + "\\NP_vkl_FZ_CAT_plan", "all", "", "input")
# arcpy.management.DeleteField(output_db + "\\NP_vkl_FZ_CAT_plan", keep_fields(output_db + "\\NP_vkl_FZ_CAT_plan", keep_fields_set))
# arcpy.analysis.Erase(output_db + "\\NP_vkl_FZ_CAT", output_db + "\\NP_vkl_FZ_CAT_plan", output_db + '\\NP_vkl_FZ_CAT_nerazgr', "")
# arcpy.env.addOutputsToMap = True
# arcpy.management.Merge([output_db + "\\NP_vkl_FZ_CAT_plan", output_db + '\\NP_vkl_FZ_CAT_nerazgr'], output_db + "\\NP_vkl_FINAL", "", "NO_SOURCE_INFO")

# arcpy.analysis.PairwiseIntersect((output_db + "\\NP_vkl_preFINAL", output_db + '\\DU'), output_db + "\\NP_vkl_DU", "all", "", "input")
# arcpy.management.DeleteField(output_db + "\\NP_vkl_DU", keep_fields(output_db + "\\NP_vkl_DU", keep_fields_set))
# arcpy.analysis.Erase(output_db + "\\NP_vkl_preFINAL", output_db + "\\NP_vkl_DU", output_db + '\\NP_vkl_DU_nerazgr', "")
# arcpy.env.addOutputsToMap = True
# arcpy.management.Merge([output_db + "\\NP_vkl_DU", output_db + '\\NP_vkl_DU_nerazgr'], output_db + "\\NP_vkl_FINAL", "", "NO_SOURCE_INFO")    

OBJECTID
SHAPE
NAME
SETTL_TYPE
Override
SHAPE_Length
SHAPE_Area


<Result 'C:\\Users\\ya.shatalov\\Desktop\\Data\\1_Проекты\\Апшеронское\\Apsheronskoe\\Perevody.gdb\\NP_vkl_FINAL'>

In [19]:
###Исключаемые ЗУ
keep_fields_set = {"DU_TYPE", "CadNumber", "cat", "bydoc", "cost", "NAME", "SETTL_TYPE", "OBJECTID", "SHAPE", "SHAPE_Length", "SHAPE_Area", "Override", "Ext_Zone_Code", "Shape", "Shape_Length", "Shape_Area"}
arcpy.env.addOutputsToMap = False
arcpy.env.workspace = "CURRENT"
arcpy.analysis.Erase(NP, NP_plan, output_db + '\\NP_iskl', "")
arcpy.management.DeleteField(output_db + '\\NP_iskl', keep_fields(output_db + '\\NP_iskl', keep_fields_set))
###ДЛЯ АППЕНДА ПОКА ЧТО ТАК
layer = output_db + "\\NP_NP_plan_inter"
target_layer = output_db + '\\NP_iskl'
prename = ''
check_fields = {'NAME': 'NAME_1', 'SETTL_TYPE': 'SETTL_TYPE_1'} 
except_fields = ['SHAPE', 'Shape', 'Shape_Length', 'Shape_Area', 'Shape.STArea()', 'Shape.STLength()'] 
fields_input = arcpy.ListFields(layer)
fields_target = arcpy.ListFields(target_layer)
fieldMappings = arcpy.FieldMappings()
for field in fields_target:
    print(field.name)
    if field.name in except_fields:
        continue
    fldMap = arcpy.FieldMap()
    field_name = field.name[field.name.rfind('.')+1:]
    if field_name in check_fields.keys():
        fldMap.addInputField(layer, check_fields[field_name])
    elif field_name in [field2.name for field2 in fields_input]:
        fldMap.addInputField(layer, field_name)
    elif field_name in [field2.name[field2.name.rfind('.')+1:] for field2 in fields_input]:
        fldMap.addInputField(layer, prename + field_name)
    else:
        continue
    targetField = fldMap.outputField
    targetField.name, targetField.aliasName, targetField.type = field.name, field.aliasName, field.type
    fldMap.outputField = targetField
    fieldMappings.addFieldMap(fldMap)
###АПЕНД ТРЕБУЕТ ЖЕРТВ
arcpy.management.Append([output_db + "\\NP_NP_plan_inter"], output_db + '\\NP_iskl', "NO_TEST", fieldMappings, expression="NAME <> NAME_1")

arcpy.analysis.PairwiseIntersect((output_db + '\\NP_iskl', ZU), output_db + "\\NP_iskl_zu", "all", "", "input")
arcpy.management.DeleteField(output_db + "\\NP_iskl_zu", keep_fields(output_db + "\\NP_iskl_zu", keep_fields_set))
arcpy.analysis.Erase(output_db + '\\NP_iskl', output_db + "\\NP_iskl_zu", output_db + '\\NP_iskl_nerazgr', "")
arcpy.management.Merge([output_db + '\\NP_iskl_nerazgr', output_db + "\\NP_iskl_zu"], output_db + "\\NP_iskl_ALL", "", "NO_SOURCE_INFO")
arcpy.analysis.PairwiseIntersect((output_db + "\\NP_iskl_ALL", FZ), output_db + "\\NP_iskl_FZ", "all", "", "input")
arcpy.management.DeleteField(output_db + "\\NP_iskl_FZ", keep_fields(output_db + "\\NP_iskl_FZ", keep_fields_set))
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.activeMap
for lyr in m.listLayers(CAT):
    lyr.definitionQuery = "STATUS = 1 And (Note <> 'Двойной учет' Or Note IS NULL)"
arcpy.analysis.PairwiseIntersect((output_db + "\\NP_iskl_FZ", CAT), output_db + "\\NP_iskl_FZ_CAT", "all", "", "input")
keep_fields_set.update(['CLASSID', 'CLASSID_1'])
arcpy.management.DeleteField(output_db + "\\NP_iskl_FZ_CAT", keep_fields(output_db + "\\NP_iskl_FZ_CAT", keep_fields_set))
for lyr in m.listLayers(CAT):
    lyr.definitionQuery = "STATUS = 2"
arcpy.analysis.PairwiseIntersect((output_db + "\\NP_iskl_FZ_CAT", CAT), output_db + "\\NP_iskl_FZ_CAT_plan", "all", "", "input")
arcpy.management.DeleteField(output_db + "\\NP_iskl_FZ_CAT_plan", keep_fields(output_db + "\\NP_iskl_FZ_CAT_plan", keep_fields_set))
arcpy.analysis.Erase(output_db + "\\NP_iskl_FZ_CAT", output_db + "\\NP_iskl_FZ_CAT_plan", output_db + '\\NP_iskl_FZ_CAT_nerazgr', "")
arcpy.management.Merge([output_db + "\\NP_iskl_FZ_CAT_plan", output_db + '\\NP_iskl_FZ_CAT_nerazgr'], output_db + "\\NP_iskl_preFINAL", "", "NO_SOURCE_INFO")

arcpy.analysis.PairwiseIntersect((output_db + "\\NP_iskl_preFINAL", output_db + '\\DU'), output_db + "\\NP_iskl_DU", "all", "", "input")
arcpy.management.DeleteField(output_db + "\\NP_iskl_DU", keep_fields(output_db + "\\NP_iskl_DU", keep_fields_set))
arcpy.analysis.Erase(output_db + "\\NP_iskl_preFINAL", output_db + "\\NP_iskl_DU", output_db + '\\NP_iskl_DU_nerazgr', "")
arcpy.env.addOutputsToMap = True
arcpy.management.Merge([output_db + "\\NP_iskl_DU", output_db + '\\NP_iskl_DU_nerazgr'], output_db + "\\NP_iskl_FINAL", "", "NO_SOURCE_INFO")    

OBJECTID
SHAPE
NAME
SETTL_TYPE
Override
SHAPE_Length
SHAPE_Area


<Result 'C:\\Users\\ya.shatalov\\Desktop\\Data\\1_Проекты\\Апшеронское\\Apsheronskoe\\Perevody.gdb\\NP_iskl_FINAL'>

In [20]:
arcpy.env.workspace = "CURRENT"
vkl_table = table_to_data_frame(output_db + "\\NP_vkl_FINAL", keep_fields(output_db + "\\NP_vkl_FINAL", keep_fields_set))
iskl_table = table_to_data_frame(output_db + "\\NP_iskl_FINAL", keep_fields(output_db + "\\NP_iskl_FINAL", keep_fields_set))
mo_table = table_to_data_frame(output_db + "\\MO_FINAL", keep_fields(output_db + "\\MO_FINAL", keep_fields_set))

In [21]:
# writer = pd.ExcelWriter('Переводы.xlsx')
# vkl_table.to_excel(writer, engine='openpyxl', sheet_name='vkl', index=False)
# iskl_table.to_excel(writer, engine='openpyxl', sheet_name='iskl', index=False)
# writer.close()

In [22]:
fz_table_file = pd.ExcelFile(fz_table_xls, 'openpyxl')

In [23]:
fz_table = pd.read_excel(fz_table_file, 'Функциональные зоны')
fz_table_file.close()

In [24]:
zone_dict = {key: value for key, value in zip(fz_table['Ext_Zone_Code'], fz_table['Zone'])}

In [25]:
category = {702010100: 'Земли населенных пунктов',
            702010200: 'Земли сельскохозяйственного назначения',
            702010300: 'Земли промышленности, энергетики, транспорта, связи, радиовещания, телевидения, информатики, земли для обеспечения космической деятельности, земли обороны, безопасности и земли иного специального назначения',
            702010400: 'Земли особо охраняемых территорий и объектов',
            702010500: 'Земли лесного фонда',
            702010800: 'Земли лесного фонда в соответствии со сведениями о земельных участках лесного фонда, сведения о которых внесены в ЕГРН',
            702010600: 'Земли водного фонда',
            702010700: 'Земли запаса'}
vkl_table = vkl_table.loc[vkl_table['Shape_Area']/10000 >= 0.005]
vkl_table['CadNumber'] = vkl_table['CadNumber'].fillna('Территории, собственность на которые не разграничена')
vkl_table['CLASSID_1'] = 702010100
vkl_table['CLASSID_1'] = vkl_table['CLASSID_1'].replace(0, 702010100)
vkl_table['cat'] = vkl_table['cat'].fillna('нет')
vkl_table['SETTL_TYPE'] = vkl_table['SETTL_TYPE'].fillna('нет')
# vkl_table['DU_TYPE'] = vkl_table['DU_TYPE'].fillna('нет')
# for i in range(len(vkl_table['DU_TYPE'])):
#     if vkl_table['DU_TYPE'].iloc[i] != 'нет':
#         vkl_table['CLASSID_1'].iloc[i] = 'Земли лесного фонда'
np = {NP_TYPE[i[0]] + ' ' + i[1]: {} for i in set(zip(vkl_table['SETTL_TYPE'], vkl_table['NAME']))}
cads = {i for i in set(vkl_table['CadNumber'].to_list())}
zu = [list(i) for i in zip(vkl_table['CadNumber'].to_list(), 
                       vkl_table['NAME'].to_list(), 
                       vkl_table['Ext_Zone_Code'].to_list(), 
                       vkl_table['cat'].to_list(), 
                       vkl_table['CLASSID'].to_list(), 
                       vkl_table['CLASSID_1'].to_list(),
                       vkl_table['Shape_Area'].to_list(),
                       vkl_table['SETTL_TYPE'].to_list())]
#                        vkl_table['DU_TYPE'].to_list())]
for i in range(len(zu)):
    if zu[i][7] != 'нет':
        zu[i][1] = NP_TYPE[zu[i][7]] + ' ' + zu[i][1]
    zu[i][4] = category[zu[i][4]]
    zu[i][5] = category[zu[i][5]]
    if zu[i][3] != 'нет':
        zu[i][4] = zu[i][3]
#     if zu[i][7] != 'нет':
#         zu[i][5] = 'Земли лесного фонда'
for i in zu:
    np[i[1]].update({i[0]:{}})
for i in zu:
    np[i[1]][i[0]].update({i[4]:[set(), set(), 0]})
for i in zu:
    np[i[1]][i[0]][i[4]][0].update([zone_dict[i[2]]])
    np[i[1]][i[0]][i[4]][1].update([i[5]])
    np[i[1]][i[0]][i[4]][2] += i[6]

result_vkl = []
for NP, ZU in np.items():
    for CAD, cat in ZU.items():
        for CAT, value in cat.items():
            result_vkl.append([CAD, NP, CAT, ', '.join(value[1]), ', '.join(value[0]), value[2]/10000])

iskl_table = iskl_table.loc[iskl_table['Shape_Area']/10000 >= 0.005]
iskl_table['CadNumber'] = iskl_table['CadNumber'].fillna('Территории, собственность на которые не разграничена')
iskl_table['CLASSID_1'] = iskl_table['CLASSID_1'].replace(0, 'нет')
iskl_table['cat'] = iskl_table['cat'].fillna('нет')
iskl_table['DU_TYPE'] = iskl_table['DU_TYPE'].fillna('нет')
iskl_table['SETTL_TYPE'] = iskl_table['SETTL_TYPE'].fillna('нет')
np = {NP_TYPE[i[0] if i[0] < 100 else i[0] - 100] + ' ' + i[1]: {} for i in set(zip(iskl_table['SETTL_TYPE'], iskl_table['NAME']))}
cads = {i for i in set(iskl_table['CadNumber'].to_list())}
zu = [list(i) for i in zip(iskl_table['CadNumber'].to_list(), 
                       iskl_table['NAME'].to_list(), 
                       iskl_table['Ext_Zone_Code'].to_list(), 
                       iskl_table['cat'].to_list(), 
                       iskl_table['CLASSID'].to_list(), 
                       iskl_table['CLASSID_1'].to_list(),
                       iskl_table['Shape_Area'].to_list(),
                       iskl_table['DU_TYPE'].to_list(),
                       iskl_table['SETTL_TYPE'].to_list())]
for i in range(len(zu)):
    if type(zu[i][8]) == int:
        if zu[i][8] > 100:
            zu[i][5] = 702010100
            zu[i][8] -= 100
    if zu[i][8] != 'нет':
        zu[i][1] = NP_TYPE[zu[i][8]] + ' ' + zu[i][1]    
    if zu[i][7] != 'нет' and zu[i][5] == 'нет':
        zu[i][5] = 702010500
    if zu[i][5] == 'нет':
        zu[i][5] = zu[i][4] 
    zu[i][4] = category[zu[i][4]]
    zu[i][5] = category[zu[i][5]]
    if zu[i][3] != 'нет':
        zu[i][4] = zu[i][3]
#     if zu[i][7] != 'нет':
#         zu[i][5] = 'Земли лесного фонда'   
for i in zu:
    np[i[1]].update({i[0]:{}})
for i in zu:
    np[i[1]][i[0]].update({i[4]:{}})
for i in zu:
    np[i[1]][i[0]][i[4]].update({i[5]: [set(), 0]})

    
for i in zu:
    np[i[1]][i[0]][i[4]][i[5]][0].update([zone_dict[i[2]]])
    np[i[1]][i[0]][i[4]][i[5]][1] += i[6]

result_iskl = []
for NP, ZU in np.items():
    for CAD, cat in ZU.items():
        for CAT, plan in cat.items():
            for PLAN, value in plan.items():
                result_iskl.append([CAD, NP, CAT, PLAN, ', '.join(value[0]), value[1]/10000])

df_vkl = pd.DataFrame(result_vkl, columns=['Кадастровый номер', 
                                           'Населенный пункт', 
                                           'Категория земель', 
                                           'Планируемая категория земель', 
                                           'Функциональные зоны', 
                                           'Площадь, га'])
df_iskl = pd.DataFrame(result_iskl, columns=['Кадастровый номер', 
                                           'Населенный пункт', 
                                           'Категория земель', 
                                           'Планируемая категория земель', 
                                           'Функциональные зоны', 
                                           'Площадь, га'])
df_vkl = df_vkl.loc[df_vkl['Площадь, га'] >= 0.005]
df_iskl = df_iskl.loc[df_iskl['Площадь, га'] >= 0.005]
###ЗА ГРАНИЦАМИ НП
mo_table = mo_table.loc[mo_table['Shape_Area']/10000 >= 0.005]
mo_table['NAME'] = 'МО'
mo_table['CLASSID_2'] = mo_table['CLASSID_1']
mo_table['CLASSID_1'] = mo_table['CLASSID']
mo_table['CLASSID'] = mo_table['CLASSID_2']
mo_table['CadNumber'] = mo_table['CadNumber'].fillna('Территории, собственность на которые не разграничена')
mo_table['cat'] = mo_table['cat'].fillna('нет')
# mo_table['DU_TYPE'] = mo_table['DU_TYPE'].fillna('нет')
np = {i:{} for i in set(mo_table['NAME'].to_list())}
cads = {i for i in set(mo_table['CadNumber'].to_list())}
zu = [list(i) for i in zip(mo_table['CadNumber'].to_list(), 
                       mo_table['NAME'].to_list(), 
                       mo_table['Ext_Zone_Code'].to_list(), 
                       mo_table['cat'].to_list(), 
                       mo_table['CLASSID'].to_list(), 
                       mo_table['CLASSID_1'].to_list(),
                       mo_table['Shape_Area'].to_list())]
#                        mo_table['DU_TYPE'].to_list())]
for i in range(len(zu)):
    zu[i][4] = category[zu[i][4]]
    zu[i][5] = category[zu[i][5]]
    if zu[i][3] != 'нет':
        zu[i][4] = zu[i][3]
#     if zu[i][7] != 'нет':
#         zu[i][5] = '702010500'

for i in zu:
    np[i[1]].update({i[0]:{}})
for i in zu:
    np[i[1]][i[0]].update({i[4]: {}})
for i in zu:
    np[i[1]][i[0]][i[4]].update({i[5]:[set(), 0]})
for i in zu:
    np[i[1]][i[0]][i[4]][i[5]][0].update([zone_dict[i[2]]])
    np[i[1]][i[0]][i[4]][i[5]][1] += i[6]

result_mo = []
for NP, ZU in np.items():
    for CAD, cat in ZU.items():
        for CAT, plan in cat.items():
            for PLAN, value in plan.items():
                result_mo.append([CAD, NP, CAT, PLAN, ', '.join(value[0]), value[1]/10000])


df_mo = pd.DataFrame(result_mo, columns=['Кадастровый номер', 
                                           'Населенный пункт', 
                                           'Категория земель', 
                                           'Планируемая категория земель', 
                                           'Функциональные зоны', 
                                           'Площадь, га'])

df_mo = df_mo.loc[df_mo['Площадь, га'] >= 0.005]

In [26]:
writer = pd.ExcelWriter(xls_name)
df_vkl.to_excel(writer, engine='openpyxl', sheet_name='Включается', index=False)
df_iskl.to_excel(writer, engine='openpyxl', sheet_name='Исключается', index=False)
df_mo.to_excel(writer, engine='openpyxl', sheet_name='За границами', index=False)
writer.close()

In [27]:
arcpy.env.workspace = output_db
for i in range(10):
    try:
        fc_Delete = list(set(arcpy.ListFeatureClasses())-{"NP_vkl_FINAL", "NP_iskl_FINAL", "MO_FINAL"})

        for fc in fc_Delete:
            if arcpy.Exists(fc):
                arcpy.Delete_management(fc,"")
    except:
        continue